In [1]:
import cudf
from pyproj import Transformer, CRS
import pandas as pd
import numpy as np
import sys,os,datetime,random
import geopandas as gpd
from shapely.geometry import Point

In [2]:
full_path='census_2020_data/census2020.csv'
small_path = 'census_2020_data/census2020_small.csv' 
small_indv_path='census_2020_data/census2020_individuals_sm.csv'

In [3]:
df = pd.read_csv(small_path,skiprows=[1],encoding='unicode_escape',usecols=['GEOCODE','STATE','STATEA','REGIONA','COUNTYA','BLOCKA','AREALAND','AREAWATR','INTPTLAT','INTPTLON','U7B001','U7B002','U7B009','U7B026','U7B047','U7B063','U7B070']).drop(0)
df.head()

,STATE,GEOCODE,REGIONA,STATEA,COUNTYA,BLOCKA,AREALAND,AREAWATR,INTPTLAT,INTPTLON,U7B001,U7B002,U7B009,U7B026,U7B047,U7B063,U7B070
1,Alabama,10379610002052,3,1,37,2052,331128,0,32.958446,-86.020502,4,4,0,0,0,0,0
2,Alabama,10950307011078,3,1,95,1078,116834,0,34.291377,-86.252923,76,71,5,0,0,0,0
3,Alabama,10890103032024,3,1,89,2024,118175,0,34.964553,-86.630753,24,23,1,1,0,0,0
4,Alabama,10279591002120,3,1,27,2120,3624159,5794,33.165842,-85.754986,44,43,1,0,0,0,0
5,Alabama,10730119013065,3,1,73,3065,3144,0,33.618408,-86.739020,0,0,0,0,0,0,0


### Random points

In [7]:
states = {1 :"AL",2 :"AK",4 :"AZ",5 :"AR",6 :"CA"}

In [8]:
df.head(2)

,STATE,GEOCODE,REGIONA,STATEA,COUNTYA,BLOCKA,AREALAND,AREAWATR,INTPTLAT,INTPTLON,U7B001,U7B002,U7B009,U7B026,U7B047,U7B063,U7B070
1,Alabama,10379610002052,3,1,37,2052,331128,0,32.958446,-86.020502,4,4,0,0,0,0,0
2,Alabama,10950307011078,3,1,95,1078,116834,0,34.291377,-86.252923,76,71,5,0,0,0,0


In [9]:
path = 'census_2020_data/tl_2021_01_tabblock20/tl_2021_01_tabblock20.shp'
gpdf = gpd.read_file(path)
gpdf.head()

,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,01,085,781100,2000,010857811002000,Block 2000,G5040,R,None,None,S,50116884,131979,+32.1235126,-086.7938124,"POLYGON ((-86.85389 32.11839, -86.85346 32.118..."
1,01,085,781200,1069,010857812001069,Block 1069,G5040,R,None,None,S,12302291,26559,+31.9871550,-086.4430289,"POLYGON ((-86.45752 31.99542, -86.45747 31.995..."
2,01,003,010200,2010,010030102002010,Block 2010,G5040,R,None,None,S,1179029,2669,+30.9176913,-087.6862008,"POLYGON ((-87.69377 30.92653, -87.69149 30.927..."
3,01,081,041002,3096,010810410023096,Block 3096,G5040,R,None,None,S,3976745,2989,+32.6964185,-085.5446355,"POLYGON ((-85.55992 32.68666, -85.55991 32.687..."
4,01,097,007204,1022,010970072041022,Block 1022,G5040,R,None,None,S,0,664897,+30.4361424,-088.1166674,"POLYGON ((-88.14153 30.40684, -88.14148 30.406..."


In [10]:
def random_points_in_polygon(number, polygon):
    points_x = np.array([])
    points_y = np.array([])
    min_x, min_y, max_x, max_y = polygon.bounds
    i= 0
    while i < number:
        point_x = random.uniform(min_x, max_x)
        point_y = random.uniform(min_y, max_y)
        if polygon.contains(Point(point_x, point_y)):
            points_x = np.append(points_x, point_x)
            points_y = np.append(points_y, point_y)
            i += 1
    return points_x, points_y # returns list of points(lat), list of points(long)

In [11]:
def generate_data(state, df_temp, gpdf):
    t1 = datetime.datetime.now()
    geoid_index_df = df_temp.index.to_numpy()
    final_points_x = np.array([])
    final_points_y = np.array([])
    geoid = np.array([])
    
    for index, row in gpdf.iterrows():
        points_x = np.array([])
        points_y = np.array([])
        geoid_temp = np.array([])
        if row['GEOID20'] in geoid_index_df and df_temp.loc[row['GEOID20']]>0:
            num_points = df_temp.loc[row['GEOID20']]
            polygon = row['geometry']
            if polygon is not None:
                points_x, points_y = random_points_in_polygon(num_points, polygon)
                geoid_temp = np.array([row['GEOID20']]*len(points_x))
                geoid = np.append(geoid,geoid_temp)
                final_points_x = np.append(final_points_x, points_x)
                # print(final_points_x)
                final_points_y = np.append(final_points_y, points_y)
                print('Processing '+str(state)+' - Completed:', "{0:0.2f}".format((index/len(gpdf))*100), '%', end='')
                print('', end='\r')

    print('Processing for '+str(state)+' complete \n total time', datetime.datetime.now() - t1)
    
    df_fin = cudf.DataFrame({'GEOID20': geoid,'x': final_points_x, 'y':final_points_y})
    df_fin.GEOID20 = df_fin.GEOID20[1:].astype('int').astype('str')
    df_fin.to_csv('census_2020_data/population_'+str(state)+'.csv', index=False)

In [14]:
def exec_data(state_key_list):
    c=0
    for i in state_key_list:
        print(i)
        c+=1
        if i< 10:
            i_str = '0'+str(i)
        else:
            i_str = str(i)
        path = 'census_2020_data/tl_2021_%s_tabblock20/tl_2021_%s_tabblock20.shp'%(i_str,i_str)
        #print(path)
        print("started reading shape file for state ", states[i])
        if os.path.isfile(path):    
            gpdf = gpd.read_file(path)[['GEOID20', 'geometry']]
            gpdf.GEOID20 = gpdf.GEOID20[1:].astype('int64')
            print("completed reading shape file for state ", states[i])
            df_temp = df.query('STATEA == @i')[['GEOCODE', 'U7B001']]
            df_temp.index = df_temp.GEOCODE
            df_temp = df_temp['U7B001']
            # print(gpdf.head(3))
            # print(gpdf)
            print("starting to generate data for "+str(states[i])+"... ")
            generate_data(states[i], df_temp, gpdf)
            del(df_temp)
        else:
            print("shape file does not exist")
            continue
        # if c==2:
        #     break

In [15]:
exec_data(states.keys())

1
started reading shape file for state  AL
completed reading shape file for state  AL
starting to generate data for AL... 
Processing for AL complete 99.87 %
 total time 0:00:04.470229
2
started reading shape file for state  AK
completed reading shape file for state  AK
starting to generate data for AK... 
Processing for AK complete 99.33 %
 total time 0:00:00.683752
4
started reading shape file for state  AZ
completed reading shape file for state  AZ
starting to generate data for AZ... 
Processing for AZ complete 98.95 %
 total time 0:00:03.860278
5
started reading shape file for state  AR
completed reading shape file for state  AR
starting to generate data for AR... 
Processing for AR complete 96.92 %
 total time 0:00:03.267842
6
started reading shape file for state  CA
completed reading shape file for state  CA
starting to generate data for CA... 
Processing for CA complete 98.54 %
 total time 0:00:13.747235


### Concat state

In [122]:
df.head(2)

,STATE,GEOCODE,REGIONA,STATEA,COUNTYA,BLOCKA,AREALAND,AREAWATR,INTPTLAT,INTPTLON,U7B001,U7B002,U7B009,U7B026,U7B047,U7B063,U7B070
1,Alabama,10379610002052,3,1,37,2052,331128,0,32.958446,-86.020502,4,4,0,0,0,0,0
2,Alabama,10950307011078,3,1,95,1078,116834,0,34.291377,-86.252923,76,71,5,0,0,0,0


In [25]:
df3 = cudf.read_csv('census_2020_data/population_%s.csv'%('AL'), usecols=['GEOID20','x', 'y'])
df3.head(3)

,GEOID20,x,y
0,<NA>,-86.728636,33.449713
1,1.073012805e+13,-86.730416,33.443373
2,1.073012805e+13,-86.732719,33.444414


In [27]:
inProj = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs' # Latitude and longitudes
outProj = 'epsg:4326' # 2D projected points
transformer = Transformer.from_crs(inProj, outProj, always_xy=True)
transformer.transform(df3['x'].to_numpy(), df3['y'].to_numpy())

(array([-96.00099526, -96.00099528, -96.00099531, ..., -96.00097886,
        -96.00097878, -96.00097888]),
 array([37.50029641, 37.50029635, 37.50029636, ..., 37.50027475,
        37.5002747 , 37.50027484]))

In [28]:
def read_state(state):
    print(state)
    print('reading '+state,end='\r')
    df2 = cudf.read_csv('census_2020_data/population_%s.csv'%(state), usecols=['GEOID20','x', 'y'])
    #print(df2)
    df2.GEOID20 = df2.GEOID20.fillna(method='bfill') # first row in every state has NA
    df2.GEOID20 = df2.GEOID20.fillna(method='ffill') # first row in every state has NA
    # print(df2[['x','y']])
    
    inProj = '+proj=aea +lat_1=29.5 +lat_2=45.5 +lat_0=37.5 +lon_0=-96 +x_0=0 +y_0=0 +datum=NAD83 +units=m +no_defs' # Latitude and longitudes
    outProj = 'epsg:4326' # 2D projected points
    transformer = Transformer.from_crs(inProj, outProj, always_xy=True)
    df2['x1'], df2['y1'] = transformer.transform(df2['x'].to_numpy(), df2['y'].to_numpy()) # Apply transformation
    # print('completed',end='\r')
    # print(df2[['x1','y1']])
    return df2

In [29]:
df.rename(columns={"GEOCODE":"GEOID20"},inplace=True)
df.head(2)

,STATE,GEOID20,REGIONA,STATEA,COUNTYA,BLOCKA,AREALAND,AREAWATR,INTPTLAT,INTPTLON,U7B001,U7B002,U7B009,U7B026,U7B047,U7B063,U7B070
1,Alabama,10379610002052,3,1,37,2052,331128,0,32.958446,-86.020502,4,4,0,0,0,0,0
2,Alabama,10950307011078,3,1,95,1078,116834,0,34.291377,-86.252923,76,71,5,0,0,0,0


In [30]:
df1 = [read_state(x) for x in list(states.values())]
final_df = cudf.concat(df1)
del(df1)
final_df = final_df.reset_index(drop=True)
final_df['p_id'] = final_df.index.astype('int32')
dataset = cudf.merge(final_df[['GEOID20','x','y','p_id']],cudf.from_pandas(df),on='GEOID20')
dataset.head()

AL
AKading AL
AZading AK
ARading AZ
CAading AR


,GEOID20,x,y,p_id,STATE,REGIONA,STATEA,COUNTYA,BLOCKA,AREALAND,AREAWATR,INTPTLAT,INTPTLON,U7B001,U7B002,U7B009,U7B026,U7B047,U7B063,U7B070
0,1.003011e+13,-87.901988,30.546257,3248,Alabama,3,1,3,1030,232758,0,30.546111,-87.900911,89,86,3,0,0,0,0
1,1.003011e+13,-87.903010,30.542832,3249,Alabama,3,1,3,1030,232758,0,30.546111,-87.900911,89,86,3,0,0,0,0
2,1.003011e+13,-87.901046,30.549183,3250,Alabama,3,1,3,1030,232758,0,30.546111,-87.900911,89,86,3,0,0,0,0
3,1.003011e+13,-87.902643,30.545547,3251,Alabama,3,1,3,1030,232758,0,30.546111,-87.900911,89,86,3,0,0,0,0
4,1.003011e+13,-87.901577,30.543567,3252,Alabama,3,1,3,1030,232758,0,30.546111,-87.900911,89,86,3,0,0,0,0


In [32]:
df3 = dataset.to_pandas()
df3.GEOID20 = df3.GEOID20.apply(lambda x: int(x))
final_data = cudf.from_pandas(df3)
final_data.head(2)

,GEOID20,x,y,p_id,STATE,REGIONA,STATEA,COUNTYA,BLOCKA,AREALAND,AREAWATR,INTPTLAT,INTPTLON,U7B001,U7B002,U7B009,U7B026,U7B047,U7B063,U7B070
0,10030112011030,-87.901988,30.546257,3248,Alabama,3,1,3,1030,232758,0,30.546111,-87.900911,89,86,3,0,0,0,0
1,10030112011030,-87.903010,30.542832,3249,Alabama,3,1,3,1030,232758,0,30.546111,-87.900911,89,86,3,0,0,0,0


In [33]:
final_data.tail()

,GEOID20,x,y,p_id,STATE,REGIONA,STATEA,COUNTYA,BLOCKA,AREALAND,AREAWATR,INTPTLAT,INTPTLON,U7B001,U7B002,U7B009,U7B026,U7B047,U7B063,U7B070
57597,60590992023006,-117.914322,33.734828,54243,California,4,6,59,3006,104138,0,33.733895,-117.914653,598,505,93,12,0,0,0
57598,60590992023006,-117.914139,33.737617,54244,California,4,6,59,3006,104138,0,33.733895,-117.914653,598,505,93,12,0,0,0
57599,60590992023006,-117.914866,33.735484,54245,California,4,6,59,3006,104138,0,33.733895,-117.914653,598,505,93,12,0,0,0
57600,60590992023006,-117.913074,33.737481,54246,California,4,6,59,3006,104138,0,33.733895,-117.914653,598,505,93,12,0,0,0
57601,60590992023006,-117.915592,33.734422,54247,California,4,6,59,3006,104138,0,33.733895,-117.914653,598,505,93,12,0,0,0


In [34]:
final_data.columns

Index(['GEOID20', 'x', 'y', 'p_id', 'STATE', 'REGIONA', 'STATEA', 'COUNTYA',
       'BLOCKA', 'AREALAND', 'AREAWATR', 'INTPTLAT', 'INTPTLON', 'U7B001',
       'U7B002', 'U7B009', 'U7B026', 'U7B047', 'U7B063', 'U7B070'],
      dtype='object')

In [36]:
# final_data.to_csv('census_2020_data/census2020_individuals_sm.csv')